In [ ]:
pip install -U transformers peft accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 87.6 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.56.1
    Uninstalling transformers-4.56.1:
      Successfully uninstalled transformers-4.56.1


In [ ]:
!pip install GPUtil

In [ ]:
import torch
import os
import GPUtil

GPUtil.showUtilization()

if torch.cuda.is_available():
  print("GPU is available")
else:
  print("GPU is not available, using CPU instead")

  os.environ["CUDA_VISIBLE_DEVICES"] = "0"
  os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"

| ID | GPU | MEM |
------------------
|  0 |  0% |  0% |
GPU is available


In [ ]:
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, LlamaTokenizer
from huggingface_hub import notebook_login
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
from datasets import load_dataset

if "COLAB_GPU" in os.environ:
  from google.colab import output
  output.enable_custom_widget_manager()

In [ ]:
if "COLAB_GPU" in os.environ:
  !huggingface-cli login
else:
  notebook_login()

⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `hf auth whoami` to get more information or `hf auth logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add t

In [ ]:
base_model_id = "meta-llama/Llama-2-7b-chat-hf"
bnb_config = BitsAndBytesConfig(
  load_in_4bit=True,                        #Quantization in 4-bit
  bnb_4bit_use_double_quant=True,           #Double Quantization --> Quantization in 4-bit then again on top of it Quantization in 4-bit
  bnb_4bit_quant_type="nf4",
  bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(base_model_id, quantization_config=bnb_config)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
!git clone https://github.com/poloclub/Fine-tuning-LLMs.git

fatal: destination path 'Fine-tuning-LLMs' already exists and is not an empty directory.


In [ ]:
train_dataset = load_dataset("text", data_files = {"train":
                                                   ["/content/Fine-tuning-LLMs/data/hawaii_wf_1.txt", "/content/Fine-tuning-LLMs/data/hawaii_wf_2.txt", "/content/Fine-tuning-LLMs/data/hawaii_wf_3.txt", "/content/Fine-tuning-LLMs/data/hawaii_wf_4.txt", "/content/Fine-tuning-LLMs/data/hawaii_wf_5.txt"]}, split="train")

In [ ]:
train_dataset["text"][0:11]

['POLICE DEPARTMENT',
 'COUNTY OF MAUI',
 '55 MAHALANI STREET',
 'WAILUKU, MAUI, HAWAII 96793',
 'TELEPHONE: (808) 244-6400',
 'FAX: (808) 244-6411',
 'JOHN PELLETIER',
 'CHIEF OF POLICE',
 'WADE M. MAEDA',
 'DEPUTY CHIEF OF POLICE',
 'During the hours of August 8, 2023, Maui became the stage for the most tragic natural disaster in state history and the deadliest fire in modem American history. ']

In [ ]:
tokenizer = LlamaTokenizer.from_pretrained(base_model_id,
                                           #use_fast is used for fast tokenizing but sometimes its not accurate so we disable it.
                                           use_fast=False,
                                           #Llama Models code is from HF so we have to say to colab that trust this code.
                                           trust_remote_code=True,
                                           #Add End of Sentence token at end
                                           add_eos_token=True)
if tokenizer.pad_token is None:
  tokenizer.add_special_tokens({'pad_token': tokenizer.eos_token})

In [ ]:
tokenized_train_dataset = []
for phrase in train_dataset:
  tokenized_train_dataset.append(tokenizer(phrase["text"]))

In [ ]:
tokenized_train_dataset[1]

{'input_ids': [1, 21122, 29979, 8079, 14861, 3120, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [ ]:
# ✅ Enable gradient checkpointing and prepare for k-bit training
model = prepare_model_for_kbit_training(model)    # prepares model for QLoRA

# ✅ Inspect available modules to confirm target layer names
# (Run once to ensure "q_proj" and "v_proj" exist for your model)
# print([name for name, _ in model.named_modules()])

# ✅ Configure LoRA
lora_config = LoraConfig(
    r=8,                       # rank
    lora_alpha=64,             # scaling factor
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],  # ⬅ adjust if your model uses different names
    lora_dropout=0.05,         # dropout for regularization
    bias="none",               # no bias tuning
    task_type="CAUSAL_LM"      # causal language modeling task
)

# ✅ Wrap the model with LoRA adapters
model = get_peft_model(model, lora_config)

In [ ]:
trainer = transformers.Trainer(
    model=model,
    train_dataset = tokenized_train_dataset,
    args=transformers.TrainingArguments(
        output_dir="./finetuneModel",
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=2,
        num_train_epochs=3,
        max_steps=100,
        learning_rate=1e-4,
        bf16=False,
        optim="paged_adamw_8bit",
        logging_dir = "./log",
        save_strategy="epoch",
        save_steps=50,
        logging_steps=10),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

model.config.use_cache = False
trainer.train()

/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,1.786900
20,1.993400
30,2.048500
40,1.966600
50,1.726000
60,1.528300
70,1.613700
80,1.582500
90,1.943100
100,2.603800


TrainOutput(global_step=100, training_loss=1.8792757129669189, metrics={'train_runtime': 797.4498, 'train_samples_per_second': 0.502, 'train_steps_per_second': 0.125, 'total_flos': 347577059106816.0, 'train_loss': 1.8792757129669189, 'epoch': 0.7079646017699115})

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig, LlamaTokenizer
from peft import PeftModel

base_model_id = "meta-llama/Llama-2-7b-chat-hf"

nf4Config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = LlamaTokenizer.from_pretrained(base_model_id, use_fast=False, trust_remote_code=True, add_eos_token=True)

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    quantization_config=nf4Config,
    device_map="auto",
    trust_remote_code=True,
    use_auth_token=True
  )

/usr/local/lib/python3.12/dist-packages/transformers/models/auto/auto_factory.py:492: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


ValueError: Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit the quantized model. If you want to dispatch the model on the CPU or the disk while keeping these modules in 32-bit, you need to set `llm_int8_enable_fp32_cpu_offload=True` and pass a custom `device_map` to `from_pretrained`. Check https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu for more details. 

In [ ]:
tokenizer = LlamaTokenizer.from_pretrained(base_model_id, use_fast=False, trust_remote_code=True, add_eos_token=True)

modelFintuned = PeftModel.from_pretrained(base_model, "finetuneModel/checkpoint-100")

RuntimeError: Error(s) in loading state_dict for PeftModelForCausalLM:
	size mismatch for base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight: copying a param with shape torch.Size([8, 4096]) from checkpoint, the shape in current model is torch.Size([8, 5120]).
	size mismatch for base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight: copying a param with shape torch.Size([4096, 8]) from checkpoint, the shape in current model is torch.Size([5120, 8]).
	size mismatch for base_model.model.model.layers.0.self_attn.k_proj.lora_A.default.weight: copying a param with shape torch.Size([8, 4096]) from checkpoint, the shape in current model is torch.Size([8, 5120]).
	size mismatch for base_model.model.model.layers.0.self_attn.k_proj.lora_B.default.weight: copying a param with shape torch.Size([4096, 8]) from checkpoint, the shape in current model is torch.Size([5120, 8]).
	size mismatch for base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight

In [ ]:
user_question = "When did Hawaii wildfiers start?"

eval_prompt = f"Question: {user_question} Just answer this question accurately and concisely..\n"

promptTokenized = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

modelFinetuned.eval()

with torch.no_grad():
  print(tokenizer.decode(modelFinetuned.generate(**promptTokenized, max_new_tokens=1024)[0], skip_special_tokens=True))
  torch.cuda.empty_cache()